'''
Data preloaded from https://www.ontario.ca/page/public-sector-salary-disclosure
for years 2008-2018
This notebook cleans data, generates gender indicator for all employees,
merges it with indeed.com ratins, creates additional calculated columns.


The resulted 2008_2018_full_data.csv format:

'sector', 'last_name', 'first_name', 'salary_x', 'benefits', 'employer',
'title', 'c_year', '_gender', '_title', 'job_category1', 'company',
'work_life', 'salary_y', 'job_security', 'management', 'culture', 'ceo',
'ceo_details', 'ctr', 'company_modified', 'company_found',
'salary_adjusted', 'benifits_adjusted', '_gender_x',
'cut_off_base_2008', '_sector', 'salary_f', 'salary_m',
'salary_adjusted_f', 'salary_adjusted_m', 'count_by_gdr_year_employer',
'male_count_year_employer', 'female_count_year_employer',
'male_year_employer', 'female_year_employer',
'female_percent_year_employer', 'salary_by_gender_year_empl',
'male_salary_year_employer', 'female_salary_year_employer',
'female_salary_percent_year_employer', 'job_category2',
'avg_indeed_score', 'num_of_ceo_ratings', 'counts_by_year_employer',
'salary_total', 'salary_adjusted_total'

'''

In [1]:
import datetime as dt
import gender_guesser.detector as gendr
import numpy as np
import pandas as pd
import time
import warnings
import operator
import timeit

pd.options.display.max_columns = None
warnings.simplefilter(action='ignore', category=FutureWarning)

data_path = "data/"

# Gender column

In [16]:
'''
Generate gender column
Two columns are created:
    _gender - 4 categories: male, female, mostly mail, mostly female
    _gender_x - 3 categories male, female, unknown

'''

df = pd.read_csv("data/merged.csv", encoding = "ISO-8859-1")
df.columns=['sector','last_name','first_name','salary','benefits','employer','title','c_year']
df.first_name = df.first_name.str.capitalize()
df.last_name = df.first_name.str.capitalize()

start_time = timeit.default_timer()
def get_gender(first_name):

    first_name = first_name.replace('-',' ')
    switcher = { 
        "mostly_female": "0.5", 
        "female": "0",
        "mostly_male": "1.5", 
        "male": "1"
    } 
    first_name = first_name.split()
    
    for i in first_name:
        return gendr.Detector().get_gender(i)

df['_gender'] = df.apply(lambda x: get_gender(x.first_name), axis=1 )

elapsed = timeit.default_timer() - start_time
print ('elapsed:', elapsed )

def com_gender(x):
    if ('female' in x):
        return 'female'
    if ('male' in x):
        return 'male'
    else:
        return 'unknown'
    
    
# df1['_gender_x'] = ['female' if 'female' in i else '' for i in df1._gender]
df['_gender_x'] = [com_gender(i) for i in df._gender]

df.to_csv(data_path + "merged_w_gen.csv", index=False)

elapsed: 8.012337200000275


# Assign job category for each job title

In [20]:
'''
Two columns are created:
    job_category1 - ceo, vp, cfo, chro, cto
    job_category2 - chief, this column overlaps with job_category1 and contains broader spectrum of jobs
'''

df = pd.read_csv(data_path + "merged_w_gen.csv")


df['_title'] = df.title.str.lower()

ceo = ["chief executive officer","ceo,","ceo ","acting chief executive officer"]
vp = ["vice president","vp ","vp, ","v.p.","acting vice president","acting vp"]
cfo = ["chief finance officer","chief financial officer"]
chro = ["chief human resources","chief human resource"]
cto = ["chief information","chief information officer","chief technology"]

chief = ["chief executive officer","ceo,","ceo ","acting chief executive officer",
         "chief finance officer","chief financial officer","chief finance officer",
         "chief financial officer","chief human resources","chief human resource",
         "chief information","chief information officer","chief technology","chief administration officer",
         "chief administrative officer","chief communications officer","chief digital officer",
         "chief development officer","chief marketing officer","chief operating officer"]

def assignCategory(f):
    for i in ceo:
        f['job_category'] = f._title.str.contains(i)
        f.loc[df['job_category'] == True, 'job_category1'] = 'ceo'
    for i in vp:
        f['job_category'] = f._title.str.contains(i)
        f.loc[df['job_category'] == True, 'job_category1'] = 'vp'
    for i in cfo:
        f['job_category'] = f._title.str.contains(i)
        f.loc[df['job_category'] == True, 'job_category1'] = 'cfo'
    for i in chro:
        f['job_category'] = f._title.str.contains(i)
        f.loc[df['job_category'] == True, 'job_category1'] = 'chro'
    for i in cto:
        f['job_category'] = f._title.str.contains(i)
        f.loc[df['job_category'] == True, 'job_category1'] = 'cto'
    for i in chief:
        f['job_category'] = f._title.str.contains(i)
        f.loc[df['job_category'] == True, 'job_category2'] = 'chief'     
    return f

df = assignCategory(df).copy()

clean = ["doctor","policy analyst","instructor","inspector","fire",
         "assistant","interim","principal","acting","associate", "deputy","library"]

for i in clean:
    df['job_category'] = df._title.str.contains(i)
    df.loc[df['job_category'] == True, 'job_category1'] = 'none'
    df.loc[df['job_category'] == True, 'job_category2'] = 'none'

df.drop(columns=['job_category'], inplace=True)
df.job_category1.fillna('none', inplace=True)
df.job_category2.fillna('none', inplace=True)
df.to_csv(data_path + "merged_w_gen_category.csv", index=False)

# Adjust salaries and benefits for Inflation

## (https://www.bankofcanada.ca/rates/related/inflation-calculator/)

In [21]:
df = pd.read_csv(data_path + "merged_w_gen_category.csv")

trans = {'1996' : 50.40,
        '1997' : 47.06,
        '1998' : 45.60,
        '1999' : 44.65,
        '2000' : 40.81,
        '2001' : 36.88,
        '2002' : 34.93,
        '2003' : 28.89 ,
        '2004' : 28.02,
        '2005' : 25.35,
        '2006' : 22.69,
        '2007' : 20.24,
        '2008' : 18.09,
        '2009' : 16.43,
        '2010' : 14.62,
        '2011' : 12.19 ,
        '2012' : 9.32,
        '2013' : 7.99 ,
        '2014' : 6.77 ,
        '2015' : 5.66 ,
        '2016' : 4.25 ,
        '2017' : 2.16,
        '2018' : 0}

def adjustForInflation(x,year):
    return round(float(x) + (float(x)/100*trans[str(year)]),2)

df['salary_adjusted_total'] = df.apply(lambda x: adjustForInflation(x.salary, x.c_year), axis=1)
df['benifits_adjusted'] = df.apply(lambda x: adjustForInflation(x.benefits, x.c_year), axis=1)

df.to_csv(data_path + "merged_w_gen_category_inflation.csv", index=False)

# Create cut-offs for each year based on inflation

This is to make sure that we compare similar data when dealing with multiple years.
For example 100,000 in 2008 corresponds to 118,090 in 2018 


In [22]:
df = pd.read_csv(data_path + "merged_w_gen_category_inflation.csv")

trans_2008 = {'2018': 118090.0,
            '2017': 116430.0,
            '2016': 114620.0,
            '2015': 112190.0,
            '2014': 109320.0,
            '2013': 107990.0,
            '2012': 106770.0,
            '2011': 105660.0,
            '2010': 104250.0,
            '2009': 102160.0,
            '2008': 100000.0}

def getCutOff(year):
    return trans_2008[str(year)]

df['cut_off_base_2008'] = df.apply(lambda x: getCutOff(x.c_year), axis=1)

df.to_csv(data_path + "merged_w_gen_category_infln_cutoff.csv", index=False)

# Clean up sector data

In [23]:
df = pd.read_csv(data_path + "merged_w_gen_category_infln_cutoff.csv")
def getSector(x):
    if (x== 'Government of Ontario : Judiciary'):
        return 'Government of Ontario - Judiciary'
    if ("Seconded" in x):
        return 'Seconded'
    if (x == 'Government of Ontario  Ministries' or x =='Government of Ontario Â Ministries' or x =='Government of Ontario : Ministries'):
        return 'Government of Ontario - Ministries'
    if (x == 'Universities - Universités'):
        return 'Universities'
    if ('Legislative Assembly' in x):
        return 'Government of Ontario - Legislative Assembly and Offices'
    if (x == 'Municipalities & Services'):
        return 'Municipalities and Services'
    if (x == 'Hospitals & Boards of Public Health'):
        return 'Hospitals and Boards of Public Health'
    if ('Colleges' in x):
        return 'Colleges'
    if (x == 'Ontario Power Generation'):
        return 'Hydro One and Ontario Power Generation'
    return x

df['_sector'] = [getSector(i) for i in df.sector]
df.to_csv(data_path + "merged_w_gen_category_infln_cutoff.csv", index=False)

# % of Female to total employees 

In [24]:
'''
1. Determine counts per year, company, gender
2. Calculate % per year and company: 
         female count / (female count + male count )* 100
'''

df = pd.read_csv(data_path + "merged_w_gen_category_infln_cutoff.csv")

# 1. counts by gender, company, year
def get_gender_counts(x):
    return x.groupby(['employer','_gender_x','c_year'])['last_name'].transform(len)
df = df.assign(count_by_gdr_year_employer=get_gender_counts)

def get_male_count(x,xx):
    if x == 'male':
        return xx
    else:
        return 0
def get_female_count(x,xx):
    if x == 'female':
        return xx
    else:
        return 0
df['male_count_year_employer'] = df.apply(lambda x: get_male_count(x._gender_x, x.count_by_gdr_year_employer), axis=1)
df['female_count_year_employer'] = df.apply(lambda x: get_female_count(x._gender_x, x.count_by_gdr_year_employer), axis=1)

def get_male(x):
    return x.groupby(['employer','c_year'])['male_count_year_employer'].transform(max)
df = df.assign(male_year_employer=get_male)

def get_female(x):
    return x.groupby(['employer','c_year'])['female_count_year_employer'].transform(max)
df = df.assign(female_year_employer=get_female)

def get_counts(x):
    return x.groupby(['employer','c_year'])['last_name'].transform(len)
df = df.assign(counts_by_year_employer=get_counts)

# 2. calculate % of female by year and company
def get_female_percent(f,m):
    if (f == 0 ):
        return 0
    if (m == 0 & f!=0):
        return 1
    return f/(f+m)
    
df['female_percent_year_employer'] = df.apply(lambda x: get_female_percent(x.female_year_employer, x.male_year_employer), axis=1)


df.to_csv(data_path + "merged_w_gen_category_infln_cutoff_fperc.csv", index=False)

# % of Female to male salary

In [27]:
'''
1. Determine avg salary per year, company, gender
2. Calculate % per year and company: 
        avg female salary / (avg female salary + avg male salary )* 100
'''

df = pd.read_csv(data_path + "merged_w_gen_category_infln_cutoff_fperc.csv")

def get_gender_salary(x):
    return x.groupby(['employer','_gender_x','c_year'])['salary'].transform(np.mean)
df = df.assign(salary_by_gdr_year_employer=get_gender_salary)

def get_male_salary(x,xx):
    if x == 'male':
        return xx
    else:
        return 0
def get_female_salary(x,xx):
    if x == 'female':
        return xx
    else:
        return 0

df['male_salary_year_employer'] = df.apply(lambda x: get_male_salary(x._gender_x, x.salary_by_gdr_year_employer), axis=1)
df['female_salary_year_employer'] = df.apply(lambda x: get_female_salary(x._gender_x, x.salary_by_gdr_year_employer), axis=1)

def get_male(x):
    return x.groupby(['employer','c_year'])['male_salary_year_employer'].transform(max)
df = df.assign(male_salary_year_employer=get_male)

def get_female(x):
    return x.groupby(['employer','c_year'])['female_salary_year_employer'].transform(max)
df = df.assign(female_salary_year_employer=get_female)


def get_female_percent(f,m):
    if (f == 0.0 ):
        return 0
    if ((m == 0.0) & (f!=0.0)):
        return 1

    return round(f/(f+m)*100,2)
    
df['female_salary_percent_year_employer'] = df.apply(lambda x: get_female_percent(x.female_salary_year_employer, x.male_salary_year_employer), axis=1)


df.to_csv(data_path + "merged_w_gen_category_infln_cutoff_fperc.csv", index=False)


# Add indeed data

In [29]:
df1 = pd.read_csv("data/companies_with_indeed_data.csv")
df = pd.read_csv("data/merged_w_gen_category_infln_cutoff_fperc.csv")

dd = pd.merge(df, df1, how='left', left_on='employer', right_on='company')
dd.drop(columns=['company'], inplace=True)
dd.to_csv(data_path + "2008_2018_full_data.csv", index=False)